In [2]:
import websocket, json , pprint , talib, numpy
from binance.client import Client
from binance.enums import *

API_KEY ='XXXX'
API_SECRET = 'XXXX'

SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30

TRADE_SYMBOL = 'ROSEUSDT'
TRADE_QUANTITY = 10



client = Client(API_KEY, API_SECRET)

closes = []
in_position = False



def on_open(ws):
    print("oppened connection")

def on_close(ws):
    print("closed connection")

def on_message(ws,message):
    #print("received message")
    json_message = json.loads(message)
    #pprint.pprint(json_message)
    candle = json_message['k']
    close = candle['c']
    is_candle_closed = candle['x']
    if is_candle_closed:
        print("candle closed :", close)
        closes.append(float(close))
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all RSIs calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current value is ", last_rsi)
            check_sell_or_buy(last_rsi)

def binance_order(symbol, side , quantity,order_type = ORDER_TYPE_MARKET):
    try:
        print("sending binance order")
        order = client.create_order(symbol=symbol , side= side,
                                    quantity = quantity, type = order_type)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

def check_sell_or_buy(last_rsi):
    global in_position

    if last_rsi > RSI_OVERBOUGHT:
        if in_position:
            print("Over bought !! SELL")
            order_statu = binance_order(TRADE_SYMBOL ,SIDE_SELL,TRADE_QUANTITY)
            if order_statu:
              in_position = False
        else:
            print("It is overbought but we dont own any!")  

    if last_rsi < RSI_OVERSOLD:
        if in_position:
            print("It is oversold but we already own it ")          
        else:
            print("Oversold ! BUY!")
            order_statu = binance_order(TRADE_SYMBOL ,SIDE_BUY,TRADE_QUANTITY)
            if order_statu:
               in_position =  True

ws = websocket.WebSocketApp(SOCKET,on_open=on_open ,on_close=on_close,on_message=on_message)
ws.run_forever()


oppened connection
candle closed : 0.03998000
[0.03998]
candle closed : 0.03998000
[0.03998, 0.03998]
candle closed : 0.04000000
[0.03998, 0.03998, 0.04]
candle closed : 0.04001000
[0.03998, 0.03998, 0.04, 0.04001]
candle closed : 0.04003000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003]
candle closed : 0.04002000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002]
candle closed : 0.04001000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001]
candle closed : 0.04003000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001, 0.04003]
candle closed : 0.04006000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001, 0.04003, 0.04006]
candle closed : 0.04005000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001, 0.04003, 0.04006, 0.04005]
candle closed : 0.04004000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001, 0.04003, 0.04006, 0.04005, 0.04004]
candle closed : 0.04007000
[0.03998, 0.03998, 0.04, 0.04001, 0.04003, 0.04002, 0.04001, 0.04003, 0.04006, 